### Import libraries

In [1]:
from pandas import read_csv
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [2]:
import statistics
import math
import glob

In [3]:
import csv

In [4]:
from scipy.stats import kurtosis as ksis

In [5]:
from sklearn.model_selection import train_test_split

In [67]:
from sklearn.metrics import roc_auc_score, confusion_matrix

In [6]:
from tqdm import tqdm

In [7]:
import math

In [8]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

### Visualizing one file

In [ ]:
example = loadmat('data/patient_3/test/patient_3_test_1054.mat')

In [ ]:
print (example['data'])

In [ ]:
samples, channels = ((example['data']).shape)

In [ ]:
print (samples)
print (channels)

In [ ]:
time = np.arange(samples)

In [ ]:
channel1 = example['data'][:,0]

In [ ]:
plt.plot(time, channel1)

In [ ]:
plt.show()

In [ ]:
def convert_file(filename, feature_extractors):
    """
    Takes in a file path, and a list of functions (feature extractors), that will be used to extract features.
    Returns a feature vector with all features for all channels
    """
    # Read data 
    data = loadmat(filename)
    samples, channels = (data['data'].shape)
    
    all_features = []
    
    # For each channel
    for c in range(channels):
        channel = data['data'][:,c]
        
        # create a list of features
        features = []
        
        # compute each desired feature
        for func in feature_extractors:
            val = func(channel)
            if (math.isnan(val)):
                features.append(0)
            else:
                features.append(val)
        
        # add this channel's features to the overall feature list of all channels
        all_features.append(features)
        
    all_features = np.array(all_features, dtype=np.float64)
        
    #for i in range(4):
    #    all_features[:,i] = (all_features[:,i] - statistics.mean(all_features[:,i])) / (max(all_features[:,i]) - min(all_features[:,i]))
        
    return (all_features)

In [ ]:
fig = plt.figure(figsize=(20,200))
for c in range(channels):
    channel = example['data'][:,c]
    plt.subplot(48,2,c+1)
    plt.plot(time, channel)
    plt.title("Channel: " + str(c))

plt.show()

In [ ]:
features_1053 = (convert_file('data/patient_3/test/patient_3_test_1054.mat', [linelength, variance, energy, kurtosis]))

In [ ]:
print (features_1053)

In [ ]:
for i in range(4):
        features_1053[:,i] = (features_1053[:,i] - statistics.mean(features_1053[:,i])) / (max(features_1053[:,i]) - min(features_1053[:,i]))
        
print (features_1053)

### Function to convert a file into a feature vector

In [49]:
def convert_file_to_sample(filename, feature_extractors):
    """
    Takes in a file path, and a list of functions (feature extractors), that will be used to extract features.
    Returns a feature vector with all features for all channels
    """
    # Read data 
    data = loadmat(filename)
    samples, channels = (data['data'].shape)
    
    all_features = []
    
    # For each channel
    for c in range(channels):
        channel = data['data'][:,c]
        
        # create a list of features
        features = []
        
        # compute each desired feature
        for func in feature_extractors:
            val = func(channel)
            if (np.isnan(val) or math.isnan(val)):
                features.append(0)
            else:
                features.append(val)
        
        # add this channel's features to the overall feature list of all channels
        all_features.append(features)
        
    all_features = np.array(all_features, dtype=np.float64)
        
    for i in range(4):
        all_features[:,i] = (all_features[:,i] - statistics.mean(all_features[:,i])) / (max(all_features[:,i]) - min(all_features[:,i]))
        
    return (np.nan_to_num(all_features))

### Feature extractors

In [50]:
def energy(second):
    energy = 0
    for i in range(second.size):
        if not math.isnan(second[i]):
            energy += second[i]**2
    energy = energy / (second.size)
    return energy

def variance(second):
    return second.var()

def linelength(second):
    ll = 0
    for i in range(second.size):
        if not (math.isnan(second[i]) or math.isnan(second[i-1])):
            ll += abs(second[i] - second[i-1])
    return ll

def kurtosis(second):
    return ksis(second)

### Experiments

1. Extract features and store in numpy array for all training data points
2. Try different classifiers for each

In [63]:
patient_id = 4

patient_str = 'patient_' + str(patient_id)
    
ictal_files = glob.glob('data/' + patient_str + '/ictal/*.mat')
nonictal_files = glob.glob('data/' + patient_str + '/non-ictal/*.mat')
#all_files = ictal_files + nonictal_files

X = []
Y = []

print ("Loading ictal files")
for filename in tqdm(ictal_files):
    X.append(convert_file_to_sample(filename, [linelength, variance, energy, kurtosis]))
    Y.append(1)
    
print ("Loading non-ictal files")
for file in tqdm(nonictal_files):
    X.append(convert_file_to_sample(filename, [linelength, variance, energy, kurtosis]))
    Y.append(0)

X = np.array(X)

Loading ictal files


100%|████████████████████████████████████████████████████████████████████████████████| 424/424 [00:26<00:00, 15.96it/s]


Loading non-ictal files


100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [01:11<00:00, 16.69it/s]


In [64]:
X = X.reshape(X.shape[0], X.shape[1]*X.shape[2])

In [65]:
X_train, X_dev, y_train, y_dev = train_test_split(X, Y, test_size=0.15, random_state=42)

In [66]:
print (X_train.shape)
print (X_dev.shape)

(1380, 352)
(244, 352)


#### Linear SVM

In [18]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-5)

In [68]:
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

In [69]:
clf.score(X_dev, y_dev)

1.0

In [70]:
predictions = clf.predict(X_dev)

In [71]:
roc_auc_score(y_dev, predictions)

1.0

In [72]:
confusion_matrix(y_dev, predictions)

array([[170,   0],
       [  0,  74]], dtype=int64)

#### Random Forest

In [73]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy')

In [74]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [75]:
predictions_forest_training = forest.predict(X_dev)

In [76]:
print ("Accuracy = ", forest.score(X_dev, y_dev))
print ("Area under ROC = ", roc_auc_score(y_dev, predictions_forest_training))
print ("Confusion Matrix: ", confusion_matrix(y_dev, predictions_forest_training))

Accuracy =  1.0
Area under ROC =  1.0
Confusion Matrix:  [[170   0]
 [  0  74]]


In [77]:
print (predictions_forest_training[:50])

[1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0
 0 0 0 0 0 0 0 0 1 0 0 1 1]


#### Logistic Regression

In [78]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()

In [79]:
logistic.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [80]:
predictions = logistic.predict(X_dev)

In [81]:
print ("Accuracy = ", logistic.score(X_dev, y_dev))
print ("Area under ROC = ", roc_auc_score(y_dev, predictions))
print ("Confusion Matrix: ", confusion_matrix(y_dev, predictions))

Accuracy =  1.0
Area under ROC =  1.0
Confusion Matrix:  [[170   0]
 [  0  74]]


In [82]:
predictions[0:10]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])

#### Save test scores

In [83]:
test_files = glob.glob('data/' + patient_str + '/test/*.mat')
print (test_files[0:10])

['data/patient_4/test\\patient_4_test_1.mat', 'data/patient_4/test\\patient_4_test_10.mat', 'data/patient_4/test\\patient_4_test_100.mat', 'data/patient_4/test\\patient_4_test_1000.mat', 'data/patient_4/test\\patient_4_test_1001.mat', 'data/patient_4/test\\patient_4_test_1002.mat', 'data/patient_4/test\\patient_4_test_1003.mat', 'data/patient_4/test\\patient_4_test_1004.mat', 'data/patient_4/test\\patient_4_test_1005.mat', 'data/patient_4/test\\patient_4_test_1006.mat']


In [84]:
len(test_files)

2184

Create feature vector of test samples

In [85]:
# Empty array
test_samples = []

# Take in feature vectors to create test dataset
for file in tqdm(test_files):
    test_samples.append(convert_file_to_sample(file, [linelength, variance, energy, kurtosis]))


100%|██████████████████████████████████████████████████████████████████████████████| 2184/2184 [02:20<00:00, 15.38it/s]


In [87]:
test_samples = test_samples.reshape(test_samples.shape[0], test_samples.shape[1]*test_samples.shape[2])

Run model on test samples

In [89]:
# Run model on test dataset
predictions_svm = clf.predict(test_samples)
predictions_forest = forest.predict(test_samples)
predictions_logistic = logistic.predict(test_samples)

#employee_writer.writerow(['John Smith', 'Accounting', 'November'])

In [90]:
print(predictions_forest[0:50])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [91]:
with open('data/' + patient_str + '/test_scores.csv', mode='w') as test_scores:
    writer = csv.writer(test_scores, delimiter=',', lineterminator='\n')
    
    writer.writerow(['id', 'prediction'])
    
    # Write to CSV
    for i in range(len(test_files)):
        writer.writerow([patient_str+'_'+str(i+1), str(predictions_forest[i])])

----------------------------------------------------------------------------------

### Training Data Pipeline

patient_1_ictal = glob.glob('data/patient_1/ictal/*.mat')
patient_1_nonictal = glob.glob('data/patient_1/non_ictal/*.mat')

y_1 = [1] * len(patient_1_ictal)
y_2 = [0] * len(patient_1_nonictal)
# x = ...
y = y1 + y2

from sklearn.utils import shuffle

def generate_samples(patient_id):
    patient_str = 'patient_' + str(patient_id)
    
    ictal_files = glob.glob('data/' + patient_str + '/ictal/*.mat')
    nonictal_files = glob.glob('data/' + patient_str + '/non-ictal/*.mat')
    all_files = ictal_files + nonictal_files
    
    y_1 = [1] * len(ictal_files)
    y_2 = [0] * len(nonictal_files)
    labels = y_1 + y_2
    
    X, y = shuffle(all_files, labels)
    
    for i in range(len(X)):
        sample = convert_file_to_sample(X[i], [linelength, variance, energy, kurtosis])
        label = y[i]
        
        yield (sample, label)

    

generator = generate_samples(1)

print (generator.__next__())